In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

In [2]:
!pip install xgboost

  Using cached https://files.pythonhosted.org/packages/96/84/4e2cae6247f397f83d8adc5c2a2a0c5d7d790a14a4c7400ff6574586f589/xgboost-0.90.tar.gz
  Stored in directory: /Users/genevievemcguire/Library/Caches/pip/wheels/e9/48/4d/de4187b5270dff71d3697c5a7857a1e2d9a0c63a28b3462eeb
Successfully built xgboost


In [3]:
# Data
import numpy as np
import pandas as pd

# Modeling
import patsy
import statsmodels.api as sm
import statsmodels.formula.api as smf
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split, GridSearchCV
#from sklearn.metrics import precision_score, recall_score, precision_recall_curve,f1_score, fbeta_score
#from sklearn.metrics import confusion_matrix

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Lasso, LassoCV, Ridge, RidgeCV
from sklearn.metrics import r2_score

# Plotting
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline 
%config InlineBackend.figure_formats = ['retina']
sns.set_style("white")

In [4]:
# Import pickled df with all data
df = pd.read_pickle('./all_data.pkl')

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58567 entries, 0 to 58566
Data columns (total 9 columns):
country_name                58567 non-null object
region_name                 58567 non-null object
income_group_name           58567 non-null object
fiscal_year                 58567 non-null int64
HDI_Change                  58567 non-null float64
assistance_category_name    58567 non-null object
implementing_agency_name    58567 non-null object
USG_sector_name             58567 non-null object
constant_amount             58567 non-null int64
dtypes: float64(1), int64(2), object(6)
memory usage: 4.0+ MB


#### Split data into data/target categories

In [6]:
X = df[['region_name', 'country_name',  
        'income_group_name', 
        'assistance_category_name', 
        'implementing_agency_name', 
        'USG_sector_name', 'constant_amount', 'fiscal_year']]
#, left out for now

y = df['HDI_Change']

#### Transform categorical variables

In [7]:
X.groupby(['country_name', 'fiscal_year'])

In [8]:
# Transform 'country_name', 'region_name', 'implementing_agency_name', 
# and 'USG_sector_name' using get_dummies

X = pd.concat([X, pd.get_dummies(X[['region_name', 'country_name', 
                                    'implementing_agency_name', 
                                    'USG_sector_name']])], axis=1)
X.drop(['country_name', 'region_name', 'implementing_agency_name', 'USG_sector_name'], axis=1, inplace=True)

In [9]:
# Change assistance_category_name to Economic_Assistance (0 or 1)
X['Economic_Assistance'] = X.assistance_category_name.apply(lambda x: 1 if x == 'Economic' else 0)

# Drop assistance_category_name column
X.drop('assistance_category_name', axis=1, inplace=True)

In [10]:
# Encode Income Group Name:
# 1 = Low Income Country
# 2 = Lower Middle Income Country
# 3 = Upper Middle Income Country
# 4 = High Income Country

def rankIncomes(incomeClass):
    if incomeClass == 'Low Income Country':
        return 0
    elif incomeClass == 'Lower Middle Income Country':
        return 1
    elif incomeClass == 'Upper Middle Income Country':
        return 2
    else:
        return 3

# Apply function and drop original column
X['Country_Income_Class'] = X.income_group_name.apply(rankIncomes)
X.drop('income_group_name', axis=1, inplace=True)

In [11]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58567 entries, 0 to 58566
Columns: 274 entries, constant_amount to Country_Income_Class
dtypes: int64(4), uint8(270)
memory usage: 16.9 MB


In [12]:
X.head()

,constant_amount,fiscal_year,region_name_East Asia and Oceania,region_name_Europe and Eurasia,region_name_Middle East and North Africa,region_name_South and Central Asia,region_name_Sub-Saharan Africa,region_name_Western Hemisphere,country_name_Afghanistan,country_name_Albania,...,USG_sector_name_Rule of Law and Human Rights,USG_sector_name_Social Assistance,USG_sector_name_Social Services,USG_sector_name_Stabilization Operations and Security Sector Reform,USG_sector_name_Trade and Investment,USG_sector_name_Transnational Crime,USG_sector_name_Tuberculosis,USG_sector_name_Water Supply and Sanitation,Economic_Assistance,Country_Income_Class
0,7053065,2003,0,0,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0
1,34632,2003,0,0,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0
2,4392782,2003,0,0,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0
3,3976353,2003,0,0,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0
4,915696,2003,0,0,0,1,0,0,1,0,...,0,0,0,1,0,0,0,0,1,0


In [13]:
# Look at feature collinearity 
#plt.figure(figsize=(200, 200))
#sns.set_context("paper")
#sns.heatmap(X.corr(), annot=False, cmap='coolwarm', vmin=-1, vmax=1)
#plt.savefig('feature_correlation.png', bbox_inches = 'tight');

### Train/Val/Test Split

In [14]:
#Split the data 60 - 20 - 20 train/val/test
X_train_val, X_test, y_train_val, y_test = train_test_split(X, y, test_size=0.2,random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=.25, random_state=43)

### Standardize

In [15]:
#After train/test split, Standardize numerical features (e.g. constant_amount)
from sklearn.preprocessing import StandardScaler

scaled_train = X_train.copy()
scaled_test = X_test.copy()

col_name = ['constant_amount']
features_train = scaled_train[col_name]
features_test = scaled_test[col_name]

scaler = StandardScaler()
train_amount = scaler.fit_transform(features_train)
test_amount = scaler.transform(features_test)

/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/data.py:645: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/anaconda3/lib/python3.7/site-packages/sklearn/base.py:464: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)
/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:13: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  del sys.path[0]


In [16]:
X_train['constant_amount'] = train_amount
X_test['constant_amount'] = test_amount

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [17]:
X_train.head()

,constant_amount,fiscal_year,region_name_East Asia and Oceania,region_name_Europe and Eurasia,region_name_Middle East and North Africa,region_name_South and Central Asia,region_name_Sub-Saharan Africa,region_name_Western Hemisphere,country_name_Afghanistan,country_name_Albania,...,USG_sector_name_Rule of Law and Human Rights,USG_sector_name_Social Assistance,USG_sector_name_Social Services,USG_sector_name_Stabilization Operations and Security Sector Reform,USG_sector_name_Trade and Investment,USG_sector_name_Transnational Crime,USG_sector_name_Tuberculosis,USG_sector_name_Water Supply and Sanitation,Economic_Assistance,Country_Income_Class
52566,-0.083203,2007,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,1
37595,-0.090869,2008,0,0,0,0,1,0,0,0,...,0,0,0,1,0,0,0,0,1,2
18259,-0.090986,2012,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,3
1604,-0.076017,2003,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,1,2
21395,-0.091449,2007,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,1,0


## Check distribution of target 

In [18]:
#histogram and normal probability plot
#from scipy.stats import norm
#sns.distplot(train['SalePrice'],fit=norm);
#fig = plt.figure()
#res = stats.probplot(train['SalePrice'], plot=plt)


## Initial OLS

In [19]:
# Create OLS model
ols_model = sm.OLS(y_train, X_train)

# Fit OLS model to training set
fit = ols_model.fit()

# Print summary statistics of the model's performance
fit.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:             HDI_Change   R-squared:                       0.170
Model:                            OLS   Adj. R-squared:                  0.164
Method:                 Least Squares   F-statistic:                     27.42
Date:                Mon, 17 Jun 2019   Prob (F-statistic):               0.00
Time:                        12:01:23   Log-Likelihood:             1.3778e+05
No. Observations:               35139   AIC:                        -2.750e+05
Df Residuals:                   34877   BIC:                        -2.728e+05
Df Model:                         261                                         
Covariance Type:            nonrobust                                         
=========================================================================================================================================
                                                                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------------------------------------------------------
constant_amount                                                        6.254e-05   2.69e-05      2.324      0.020    9.79e-06       0.000
fiscal_year                                                              -0.0001   5.18e-06    -21.288      0.000      -0.000      -0.000
region_name_East Asia and Oceania                                         0.1467      0.007     21.811      0.000       0.134       0.160
region_name_Europe and Eurasia                                            0.1364      0.006     21.790      0.000       0.124       0.149
region_name_Middle East and North Africa                                  0.1378      0.006     21.478      0.000       0.125       0.150
region_name_South and Central Asia                                        0.1504      0.007     21.937      0.000       0.137       0.164
region_name_Sub-Saharan Africa                                            0.1635      0.007     21.799      0.000       0.149       0.178
region_name_Western Hemisphere                                            0.1421      0.007     21.618      0.000       0.129       0.155
country_name_Afghanistan                                                  0.0276      0.001     22.144      0.000       0.025       0.030
country_name_Albania                                                      0.0130      0.001     21.474      0.000       0.012       0.014
country_name_Algeria                                                      0.0113      0.001     19.419      0.000       0.010       0.012
country_name_Angola                                                      -0.0097      0.001    -12.448      0.000      -0.011      -0.008
country_name_Antigua and Barbuda                                         -0.0117      0.001    -10.632      0.000      -0.014      -0.010
country_name_Argentina                                                    0.0047      0.000     11.062      0.000       0.004       0.006
country_name_Armenia                                                      0.0258      0.001     21.466      0.000       0.023       0.028
country_name_Australia                                                   -0.0149      0.001    -11.446      0.000      -0.017      -0.012
country_name_Austria                                                     -0.0047      0.002     -3.133      0.002      -0.008      -0.002
country_name_Azerbaijan                                                   0.0132      0.001     21.544      0.000       0.012       0.014
country_name_Bahamas                                                     -0.0112      0.001    -15.397      0.000      -0.013      -0.010
country_name_Bahrain                           

## LassoCV

In [20]:
# Run cross validation, find the best alpha, refit the model on all the data with that alpha
alphavec = 10**np.linspace(-4,4)

lasso_model = LassoCV(alphas = alphavec, cv=5)
lasso_model.fit(X_train, y_train)

# Best alpha value:
lasso_model.alpha_

0.0001

In [21]:
# These are the (standardized) coefficients found when it refit using that best alpha
#list(zip(X_train.columns, lasso_model.coef_))

In [22]:
# Make predictions on the test set using the model
test_set_pred = lasso_model.predict(X_test)

# Evaluation:
r2_score(y_test, test_set_pred)

0.037757376968837075

## RidgeCV

In [23]:
ridge_model = RidgeCV(alphas = alphavec, cv=5)
ridge_model.fit(X_train, y_train)

list(zip(X_train.columns, ridge_model.coef_))

[('constant_amount', 6.247923342500163e-05),
 ('fiscal_year', -0.00011064950878317746),
 ('region_name_East Asia and Oceania', 0.0005424809813637975),
 ('region_name_Europe and Eurasia', 0.0002530300401826856),
 ('region_name_Middle East and North Africa', -0.0016695613419315822),
 ('region_name_South and Central Asia', 0.001332677155549341),
 ('region_name_Sub-Saharan Africa', 0.0003976043629474241),
 ('region_name_Western Hemisphere', -0.000856231198105522),
 ('country_name_Afghanistan', 0.0011404164016016044),
 ('country_name_Albania', 0.0016317398206008276),
 ('country_name_Algeria', 0.0030813404837511525),
 ('country_name_Angola', 0.005710965837569404),
 ('country_name_Antigua and Barbuda', -0.0020009065073738116),
 ('country_name_Argentina', 5.18117947543071e-05),
 ('country_name_Armenia', 0.00038384585864769197),
 ('country_name_Australia', -0.0018373136055135396),
 ('country_name_Austria', -0.0015375315540973756),
 ('country_name_Azerbaijan', 0.001770818629727351),
 ('country_n

In [24]:
# Make predictions on the test set using the model
test_set_pred = ridge_model.predict(X_test)

# Evaluation:
r2_score(y_test, test_set_pred)

0.17175821864144925

## RandomForestRegressor

In [25]:
#from sklearn.model_selection import cross_val_score, GridSearchCV
#from sklearn.ensemble import RandomForestRegressor
#from sklearn.preprocessing import MinMaxScaler

In [26]:
#gsc = GridSearchCV(estimator=RandomForestRegressor(), 
#                   param_grid={
#                       'max_depth': range(3,7), 
#                       'n_estimators': (10, 50, 100, 1000)}, 
#                   cv=5, scoring='neg_mean_squared_error', 
#                   verbose=0, n_jobs=-1)

#grid_result = gsc.fit(X_train, y_train)
#best_params = grid_result.best_params_

In [27]:
#rfr_model = RandomForestRegressor(max_depth=best_params["max_depth"], 
#                                  n_estimators=best_params["n_estimators"], 
#                                  random_state=42, verbose=False)

#rfr_model.fit(X_train, y_train)

In [28]:
# Make predictions on the test set using the model
#rfr_test_set_pred = rfr_model.predict(X_test)

# Evaluation:
#r2_score(y_test, rfr_test_set_pred)

## XGBoost

In [29]:
from sklearn.metrics import accuracy_score
import xgboost as xgb

In [30]:
#Evaluate models with Root Mean Squared Error
#def rmse(actuals, preds):
#    return np.sqrt(((actuals - preds) ** 2).mean())

In [31]:
gbm = xgb.XGBRegressor(n_estimators=3000, #arbitrary large number
                       max_depth=3,
                       objective="reg:linear",
                       learning_rate=.1, 
                       subsample=1,
                       min_child_weight=1,
                       colsample_bytree=.8)

In [32]:
eval_set=[(X_train, y_train),(X_val, y_val)] #tracking train/validation error as we go
fit_model = gbm.fit(X_train, y_train)#, 
     #               eval_set=eval_set,
   #                 eval_metric='rmse',
    #                early_stopping_rounds=20,
    #                verbose=True 
    #               )


/anaconda3/lib/python3.7/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


[12:02:09] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[0]	validation_0-rmse:0.444983	validation_1-rmse:0.444967
Multiple eval metrics have been passed: 'validation_1-rmse' will be used for early stopping.

Will train until validation_1-rmse hasn't improved in 20 rounds.
[1]	validation_0-rmse:0.400482	validation_1-rmse:0.400476
[2]	validation_0-rmse:0.360456	validation_1-rmse:0.360438
[3]	validation_0-rmse:0.324418	validation_1-rmse:0.324402
[4]	validation_0-rmse:0.29198	validation_1-rmse:0.291964
[5]	validation_0-rmse:0.262799	validation_1-rmse:0.262777
[6]	validation_0-rmse:0.236512	validation_1-rmse:0.236512
[7]	validation_0-rmse:0.21288	validation_1-rmse:0.212875
[8]	validation_0-rmse:0.191606	validation_1-rmse:0.191599
[9]	validation_0-rmse:0.172468	validation_1-rmse:0.172453
[10]	validation_0-rmse:0.155247	validation_1-rmse:0.155223
[11]	validation_0-rmse:0.139732	validation_1-rmse:0.139722
[12]	validation_0-rmse:0.1257

[135]	validation_0-rmse:0.004341	validation_1-rmse:0.004343
[136]	validation_0-rmse:0.004339	validation_1-rmse:0.004341
[137]	validation_0-rmse:0.004335	validation_1-rmse:0.004337
[138]	validation_0-rmse:0.004333	validation_1-rmse:0.004338
[139]	validation_0-rmse:0.004332	validation_1-rmse:0.004337
[140]	validation_0-rmse:0.004331	validation_1-rmse:0.004336
[141]	validation_0-rmse:0.004325	validation_1-rmse:0.00433
[142]	validation_0-rmse:0.004324	validation_1-rmse:0.004327
[143]	validation_0-rmse:0.00432	validation_1-rmse:0.004324
[144]	validation_0-rmse:0.004315	validation_1-rmse:0.00432
[145]	validation_0-rmse:0.004303	validation_1-rmse:0.004312
[146]	validation_0-rmse:0.004303	validation_1-rmse:0.004312
[147]	validation_0-rmse:0.004301	validation_1-rmse:0.00431
[148]	validation_0-rmse:0.004299	validation_1-rmse:0.004309
[149]	validation_0-rmse:0.004298	validation_1-rmse:0.004308
[150]	validation_0-rmse:0.004294	validation_1-rmse:0.004303
[151]	validation_0-rmse:0.00429	validation_1

[273]	validation_0-rmse:0.004078	validation_1-rmse:0.004106
[274]	validation_0-rmse:0.004078	validation_1-rmse:0.004106
[275]	validation_0-rmse:0.004077	validation_1-rmse:0.004104
[276]	validation_0-rmse:0.004076	validation_1-rmse:0.004103
[277]	validation_0-rmse:0.004074	validation_1-rmse:0.004102
[278]	validation_0-rmse:0.004073	validation_1-rmse:0.004101
[279]	validation_0-rmse:0.004072	validation_1-rmse:0.0041
[280]	validation_0-rmse:0.004071	validation_1-rmse:0.004099
[281]	validation_0-rmse:0.00407	validation_1-rmse:0.004098
[282]	validation_0-rmse:0.004069	validation_1-rmse:0.004098
[283]	validation_0-rmse:0.004068	validation_1-rmse:0.004096
[284]	validation_0-rmse:0.004066	validation_1-rmse:0.004094
[285]	validation_0-rmse:0.004065	validation_1-rmse:0.004093
[286]	validation_0-rmse:0.004065	validation_1-rmse:0.004093
[287]	validation_0-rmse:0.004065	validation_1-rmse:0.004093
[288]	validation_0-rmse:0.004064	validation_1-rmse:0.004093
[289]	validation_0-rmse:0.004063	validation

In [34]:
fit_model_pred = fit_model.predict(X_test)

r2_score(y_test, fit_model_pred)

0.41137055895376684

In [ ]:
#To tune, we should use validation results and ignore test until 
#final verification. So here's the validation error benchmark we want to beat:
#rmse(gbm.predict(X_val, ntree_limit=gbm.best_ntree_limit),y_val) 

In [ ]:
#### Standard Scaling

#from sklearn.pipeline import Pipeline
#from sklearn.preprocessing import StandardScaler

## This step fits the Standard Scaler to the training data
## Essentially it finds the mean and standard deviation of each variable in the training set

#std = StandardScaler()
#std.fit(X_train.values)

## This step applies the scaler to the train set.
## It subtracts the mean it learned in the previous step and then divides by the standard deviation

#X_tr = std.transform(X_train.values)

## Apply the scaler to the test set

#X_te = std.transform(X_test.values)